In [1]:
%pwd

'E:\\Github Projects\\Retinal_Blood_Vessels_Segmentation\\notebooks'

In [2]:
import sys
sys.path.append("E:\\Github Projects\\Retinal_Blood_Vessels_Segmentation")

In [3]:
import cv2
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.under_sampling import RandomUnderSampler, NearMiss

from scripts.prepare_data import create_dataset_from_directory

In [4]:
X_train, y_train = create_dataset_from_directory(dir="../images/train",
                                                 channel=1,
                                                 shape=None,
                                                 window_shape=(5, 5),
                                                 pad=True,
                                                 padding=(2, 2))

y_train = (y_train/255).astype(int)

Image 1/21
Image 2/21
Image 3/21
Image 4/21
Image 5/21
Image 6/21
Image 7/21
Image 8/21
Image 9/21
Image 10/21
Image 11/21
Image 12/21
Image 13/21
Image 14/21
Image 15/21
Image 16/21
Image 17/21
Image 18/21
Image 19/21
Image 20/21
Image 21/21


In [5]:
X_train.shape, y_train.shape, np.unique(y_train, return_counts=True)

((20139840, 32),
 (20139840,),
 (array([0, 1]), array([18710253,  1429587], dtype=int64)))

In [ ]:
X_train = PCA(n_components=0.99).fit_transform(X_train)

In [ ]:
X_train.shape

In [6]:
undersampler = RandomUnderSampler(random_state=42)
# undersampler = NearMiss()

X_undersampled, y_undersampled = undersampler.fit_resample(X_train, y_train)

del X_train
del y_train

In [7]:
X_undersampled.shape, y_undersampled.shape, np.unique(y_undersampled, return_counts=True)

((2859174, 32),
 (2859174,),
 (array([0, 1]), array([1429587, 1429587], dtype=int64)))

In [8]:
pipeline = Pipeline(steps=[('scaler', StandardScaler()),
                           ('clf', RandomForestClassifier(random_state=42))])

params = [
    {
        "clf__max_depth": np.arange(6, 11, 2),
        "clf__n_estimators": np.arange(20, 61, 10),
        "scaler": [MinMaxScaler(), StandardScaler()]
    }
]

grid = GridSearchCV(pipeline,
                    params,
                    cv=4,
                    verbose=3,
                    scoring=['f1', 'precision', 'recall'],
                    refit='f1')

grid.fit(X_undersampled, y_undersampled)

Fitting 4 folds for each of 30 candidates, totalling 120 fits
[CV 1/4] END clf__max_depth=6, clf__n_estimators=20, scaler=MinMaxScaler(); f1: (test=0.754) precision: (test=0.615) recall: (test=0.974) total time= 1.5min
[CV 2/4] END clf__max_depth=6, clf__n_estimators=20, scaler=MinMaxScaler(); f1: (test=0.759) precision: (test=0.618) recall: (test=0.984) total time= 1.6min
[CV 3/4] END clf__max_depth=6, clf__n_estimators=20, scaler=MinMaxScaler(); f1: (test=0.757) precision: (test=0.617) recall: (test=0.977) total time= 1.5min
[CV 4/4] END clf__max_depth=6, clf__n_estimators=20, scaler=MinMaxScaler(); f1: (test=0.713) precision: (test=0.601) recall: (test=0.876) total time= 1.5min
[CV 1/4] END clf__max_depth=6, clf__n_estimators=20, scaler=StandardScaler(); f1: (test=0.755) precision: (test=0.615) recall: (test=0.976) total time= 1.4min
[CV 2/4] END clf__max_depth=6, clf__n_estimators=20, scaler=StandardScaler(); f1: (test=0.759) precision: (test=0.618) recall: (test=0.984) total time=

KeyboardInterrupt: 